### End to End demo of starting spot instance in AWS

In [54]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [55]:
from aws_setup import *

### Testing

In [56]:
list(ec2.key_pairs.all())

[ec2.KeyPairInfo(name='AmazonEC2test'),
 ec2.KeyPairInfo(name='aws-key-fast-ai'),
 ec2.KeyPairInfo(name='aws-key-test-vpc'),
 ec2.KeyPairInfo(name='id_rsa'),
 ec2.KeyPairInfo(name='Mac')]

#### Define parameters

In [70]:
vpc_name='fast-ai'
instance_name = f'{vpc_name}-instance'
instance_type = 't2.medium'

#### Get Existing VPC by tag name

In [71]:
vpc = get_vpc(vpc_name); vpc

ec2.Vpc(id='vpc-6e6b2a17')

#### Request Spot instance

In [23]:
spot_prices = get_spot_prices(); spot_prices[instance_type]

'0.031900'

In [24]:
launch_specs = LaunchSpecs(vpc, instance_type=instance_type, volume_size=80).build()

In [26]:
launch_specs

{'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
   'Ebs': {'DeleteOnTermination': True,
    'VolumeSize': 80,
    'VolumeType': 'gp2'}}],
 'ImageId': 'ami-8c4288f4',
 'InstanceType': 't2.medium',
 'KeyName': 'aws-key-fast-ai',
 'NetworkInterfaces': [{'AssociatePublicIpAddress': True,
   'DeviceIndex': 0,
   'Groups': ['sg-f60fca88'],
   'SubnetId': 'subnet-f056ff89'}]}

In [27]:
instance = create_spot_instance(instance_name, launch_specs, spot_price='0.5'); instance

Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-07f29a1137e5bd67f
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-fast-ai.pem ubuntu@34.217.9.117


ec2.Instance(id='i-07f29a1137e5bd67f')

Request on demand instance (if spot error)

In [ ]:
# instance = create_instance(instance_name, vpc, instance_type='t2.micro'); instance

In [72]:
instance = get_instance(instance_name); instance

ec2.Instance(id='i-0976e3eb46f705bb0')

In [73]:
v1 = list(instance.volumes.all())[0]

In [79]:
v1.detach_from_instance()

ClientError: An error occurred (IncorrectState) when calling the DetachVolume operation: Volume 'vol-0ec18e0d7a4782ee8'is in the 'available' state.

In [86]:
v1.state

'in-use'

### SSH

In [34]:
client = connect_to_instance(instance)

Connecting to SSH...
Connected!


#### Attach EBS volume (Optional)

In [68]:
volume_tag = 'fast-ai-imagenet'

In [36]:
_ = attach_volume(instance, volume_tag, device='/dev/xvdf')

In [42]:
mount_volume(instance, device='/dev/xvdf', mount_dir='ebs_mount')

Connecting to SSH...
Connected!


#### Mount EFS

In [48]:
efs_addr = get_efs_address('fast-ai-efs'); efs_addr

'fs-0ea233a7.efs.us-west-2.amazonaws.com'

In [46]:
out, _ = run_command(client, 'mkdir ~/efs_mount_point')

run_command returned: 



In [49]:
efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount_point'

In [50]:
out, _ = run_command(client, efs_mount_cmd)

run_command returned: 



In [51]:
out, _ = run_command(client, 'ls efs_mount_point') # no reformatting

run_command returned: 
efs_saved.txt

